# Convolutional Neural Network Using Mel Spectrogram Classifying Gender

In [7]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.regularizers import l2
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

%matplotlib inline

In [8]:
audio_data = np.load("../data/audio_data.npy")
labels = np.load("../data/wav_labels.npy")

# labels: modality-vocal channel-emotion-emotional intensity-statement-repetition-actor
# emotions: 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# odd number actors = male, even = female

# 1440 files: 24 speakers, 60 recordings per speaker
audio_data = audio_data.reshape(1440, 9480)

features = []

for i in range(1440):
    
    if (labels[i][6]%2 == 0):
        label = 1
    else:
        label = 0
    
    features.append([audio_data[i], label])
    
feature_df = pd.DataFrame(features, columns = ["mfcc", "gender"])

feature_df.head()


,mfcc,gender
0,"[-710.0553588867188, -709.9026489257812, -711....",1
1,"[-547.765625, -548.0353393554688, -548.6129760...",1
2,"[-616.4595336914062, -615.7564697265625, -615....",1
3,"[-739.8626098632812, -738.7739868164062, -735....",1
4,"[-698.0630493164062, -697.3838500976562, -696....",1


In [9]:
## ===== Males ===== ##
X = np.array(feature_df.mfcc.tolist())
y = np.array(feature_df.gender.tolist())

#20-80 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
X_train.shape

(1152, 9480)

In [10]:
# Reshape for CNN input
X_train = np.array([x.reshape( (20, 474, 1) ) for x in X_train])
X_test = np.array([x.reshape( (20, 474, 1) ) for x in X_test])

# One-Hot encoding for classes
y_train = np.array(to_categorical(y_train, 2))
y_test = np.array(to_categorical(y_test, 2))

In [11]:
model = Sequential()

model.add(Conv2D(64, (5,5), input_shape=X_test.shape[1:], kernel_regularizer=l2(l=0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(24, (4,4)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(rate=0.2))

model.add(Conv2D(24, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(rate=0.2))

model.add(Conv2D(80, (2,2)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(rate=0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(2))
model.add(Activation('softmax'))

In [12]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy'])

model.fit(
	x=X_train,
	y=y_train,
    epochs=100,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
9/9 [==============================] - 0s 55ms/step - loss: 13.8479 - accuracy: 0.4948 - val_loss: 0.7116 - val_accuracy: 0.4861
Epoch 2/100
9/9 [==============================] - 0s 45ms/step - loss: 0.7253 - accuracy: 0.5035 - val_loss: 0.7129 - val_accuracy: 0.4861
Epoch 3/100
9/9 [==============================] - 0s 45ms/step - loss: 0.7104 - accuracy: 0.5035 - val_loss: 0.6981 - val_accuracy: 0.4861
Epoch 4/100
9/9 [==============================] - 0s 44ms/step - loss: 0.6762 - accuracy: 0.5035 - val_loss: 0.6734 - val_accuracy: 0.4861
Epoch 5/100
9/9 [==============================] - 0s 45ms/step - loss: 0.6499 - accuracy: 0.6580 - val_loss: 0.6604 - val_accuracy: 0.7361
Epoch 6/100
9/9 [==============================] - 0s 45ms/step - loss: 0.6229 - accuracy: 0.7214 - val_loss: 0.6157 - val_accuracy: 0.7535
Epoch 7/100
9/9 [==============================] - 0s 44ms/step - loss: 0.5887 - accuracy: 0.7569 - val_loss: 0.6264 - val_accuracy: 0.7361
Epoch 8/100
9/9 [==